## A GeneratorContextManager Example

A short code example taken from an article on the O'Reilly website.

https://www.oreilly.com/learning/20-python-libraries-you-arent-using-but-should?imm_mid=0e7ab9&cmp=em-prog-na-na-newsltr_20160910

20 Python libraries you aren't using (but should):  
Discover lesser-known Python libraries that are easy to install and use, cross-platform, and applicable to more than one domain.

By Caleb Hattingh
September 1, 2016

Hattingh's discussion of the code example is included in his article, but the following discussion tries to delve a bit deeper for those still learning.


First, a ContextManager is defined using the @contextmanager decorator.

In [1]:
from time import perf_counter
from array import array
from contextlib import contextmanager

@contextmanager  
def timing(label: str):
    t0 = perf_counter()  
    yield lambda: (label, t1 - t0)  
    t1 = perf_counter() 

The *timing* context manager features two calls to *time.perf_counter()* to calculate a time interval of short duration in fractional seconds.  These are essentially high-precision timestamps from an external clock, so there is no concern when confronting overlapping usage. 

Under the covers, a ContextManager has an *enter()* method to build up the context and an *exit()* method to tear it down.  This is commonly understood when opening a file using the *with* syntax and having the ContextManager open the file and close it automatically at the end.  Using the @contextmanager, the code before the *yield* prepares the context and the code after it tears it down.

The *yield* indicates that *timing* is a generator.  It yields a lambda function that returns a tuple with the assigned label for a given *timing* instance and the corresponding calculation of the time interval.  Note that the sequential nature of the code that places the calculation *t1 - t0* above the assignment of *t1* forces us to see that the yielded lambda is deferred and that we are dealing with a closure.  The lambda retains access to t0 and t1 after it is returned to the caller and when the lambda is called, as shown later on, both perf_counter values are available.

Inspecting an instance of the type:

In [2]:
timing("xx")

The rest of Hattingh's code uses the *timing* ContextManager to time two sequential array operations and embeds them in another instance of *timer* to calculate a total.  As Hattingh notes, the second method of array creation is significantly faster.  The total amount is slightly longer that the sum of the inner two, as one might expect for some overhead.

In [3]:
with timing('Array tests') as total:  
    with timing('Array creation innermul') as inner:
        x = array('d', [0] * 1000000)  

    with timing('Array creation outermul') as outer:
        x = array('d', [0]) * 1000000 


print('Total [%s]: %.6f s' % total())
print('    Timing [%s]: %.6f s' % inner())
print('    Timing [%s]: %.6f s' % outer())


Total [Array tests]: 0.051668 s
    Timing [Array creation innermul]: 0.050380 s
    Timing [Array creation outermul]: 0.001255 s


Each of the instances of *timing* is associated with a variable name:  "total", "inner", and "outer" following the *as* in each with statement.  Each of these names is a reference to the yielded lambda closure, the means of access to the perf_counter calculation and the label.  

In [4]:
outer()

('Array creation outermul', 0.0012550149986054748)

Each of these functions is static, returning the same values every time it is called since the perf_counter calls were done once per contextmanager.

It is possible to inspect these function objects as to how the closures are managed internally. The label, t0 and t1 values are stored as cell contents, but not the result of the subtraction.

In [5]:
(outer.__closure__[0].cell_contents, 
 outer.__closure__[1].cell_contents, 
 outer.__closure__[2].cell_contents)

('Array creation outermul', 26942.765028169, 26942.766283184)

Hattingh is slightly apologetic for being "a little clever"; the underlying sense of the code may not be immediately obvious.  The elegance of Python here can be contrasted with other languages that would be overt in expressing structures in a more declamatory fashion.

Additional notes on using the @contextmanager decorator:  

* In recipe 43 of Brett Slatkin's book *Effective Python*, the definition of the contextmanager with a decorator is demonstrated and in his example wraps the yield with "try:" and the remaining secion with "finally:".  In the example above, this would have no effect since there is no cleanup and if the yield failed the t1 assignment is unimportant.
 
* Chapter 15 of Luciano Ramalho's *Fluent Python* has a detailed discussion of contextmanagers.
 